In [1]:
!pip install neptune-client

     |████████████████████████████████| 92kB 3.8MB/s 
     |████████████████████████████████| 829kB 9.5MB/s 
     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 204kB 29.9MB/s 
     |████████████████████████████████| 163kB 31.7MB/s 
     |████████████████████████████████| 143kB 27.7MB/s 
     |████████████████████████████████| 133kB 42.1MB/s 
     |████████████████████████████████| 71kB 6.3MB/s 
     |████████████████████████████████| 71kB 6.3MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.4.119-py2.py3-none-any.whl size=150019 sha256=a6699533c81adb616d4ccd63c7477ac87d6a60bc851e00d0bb774b5e2d65e198
  Stored in directory: /root/.cache/pip/wheels/84/5c/c1/a81e80761b94b4467fd3fda1fd3109463702f6247fc422eb33
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=1cb12b8d8f651b3d378bd7efa4f6c7da95c4f936b2afba6f03c2cdb412e1416c
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a93

In [61]:
import neptune
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re
sns.set(style="darkgrid")

project = neptune.init(
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNGIzMTllNDQtZGVjMi00MjcxLThiOTAtYTcxNzJiNWI5NjkzIn0=",
            project_qualified_name="elangenhan/hcd-experiments"
        )

In [104]:
experiments = project.get_experiments()

logs_names_deprecated = ['dur', 'train_loss', 'train_acc', 'train_f1', 'train_roc_auc', 'train_precision', 'train_recall', 'test_acc' , 'test_f1' , 'test_roc_auc' , 'test_precision' , 'test_recall']
logs_names            = ['dur', 'train_loss', 'train_acc', 'train_f1', 'train_roc_auc', 'train_precision', 'train_recall', 'valid_acc', 'valid_f1', 'valid_roc_auc', 'valid_precision', 'valid_recall', 'valid_loss']


counter = 1
length = len(experiments)
metrics_df = pd.DataFrame(columns=['id', *logs_names])
for experiment in experiments:
  print(f'{counter}/{length} - {experiment.id} \r')
  counter = counter + 1
  if experiment.state == 'succeeded' and experiments[46].get_logs()['dur']['lastX'] == 30.0:
    if int(experiment.id.split('-')[1]) < 114:
      df = experiment.get_numeric_channels_values(*logs_names_deprecated)
      df = df.rename(columns={'test_acc': 'valid_acc', 'test_f1': 'valid_f1', 'test_roc_auc': 'valid_roc_auc', 'test_precision': 'valid_precision', 'test_recall': 'valid_recall'})
      df.insert(loc=12, column='valid_loss', value="")
    else:
      df = experiment.get_numeric_channels_values(*logs_names)

    # net_type vgg 11 etc net + number
    net_type = experiment.get_parameters()["module"].split('\'')[1].split('.')[1].split('Pretrained')[0]
    df.insert(loc=13, column='net_type', value=net_type)

    # net_family vgg only net
    match = re.match(r"([a-z]+)([0-9]+)", experiments[0].name, re.I)
    if match: items = match.groups()
    df.insert(loc=13, column='net_family', value=items[0])

    # pretrained - boolean 0 oder 1 if pretrained
    pretrained = 0
    if "pretrained" in experiment.name.lower(): pretrained = 1
    df.insert(loc=14, column='pretrained', value=pretrained)

    # normalized - boolean 0 oder 1 if normalized
    normalized = 0
    if "normalized" in experiment.name.lower(): normalized = 1
    df.insert(loc=15, column='normalized', value=normalized)

    # full_data - boolean 0 oder 1 if full data set used
    full_data = 1
    if hasattr(experiment.get_logs(), 'test_name'):
      if "small" in experiment.get_logs()['test_name'].y: full_data = 0
    else:
      if "small" in experiment.name: full_data = 0

    df.insert(loc=16, column='full_data', value=full_data)

    df.insert(loc=0, column='id', value=experiment.id)
    metrics_df = metrics_df.append(df, sort=True)

metrics_df.to_csv (r'./neptune_logs.csv', index = False, header=True)
print("done saving")

1/102 - HCDEX-17 
2/102 - HCDEX-18 
3/102 - HCDEX-19 
4/102 - HCDEX-20 
5/102 - HCDEX-21 
6/102 - HCDEX-22 
7/102 - HCDEX-24 
8/102 - HCDEX-53 
9/102 - HCDEX-54 
10/102 - HCDEX-60 
11/102 - HCDEX-61 
12/102 - HCDEX-66 
13/102 - HCDEX-68 
14/102 - HCDEX-69 
15/102 - HCDEX-70 
16/102 - HCDEX-71 
17/102 - HCDEX-72 
18/102 - HCDEX-73 
19/102 - HCDEX-74 
20/102 - HCDEX-75 
21/102 - HCDEX-77 
22/102 - HCDEX-78 
23/102 - HCDEX-79 
24/102 - HCDEX-80 
25/102 - HCDEX-81 
26/102 - HCDEX-84 
27/102 - HCDEX-85 
28/102 - HCDEX-87 
29/101 - HCDEX-89 
30/101 - HCDEX-90 
31/101 - HCDEX-91 
32/101 - HCDEX-92 
33/101 - HCDEX-94 
34/101 - HCDEX-96 
35/101 - HCDEX-97 
36/101 - HCDEX-98 
37/101 - HCDEX-99 
38/101 - HCDEX-100 
39/101 - HCDEX-101 
40/101 - HCDEX-102 
41/101 - HCDEX-103 
42/101 - HCDEX-104 
43/101 - HCDEX-105 
44/101 - HCDEX-106 
45/101 - HCDEX-107 
46/101 - HCDEX-108 
47/101 - HCDEX-109 
48/101 - HCDEX-110 
49/101 - HCDEX-111 
50/101 - HCDEX-112 
51/101 - HCDEX-113 
52/100 - HCDEX-125 
53/99 

In [103]:
experiments[46].get_logs()['dur']['lastX']

30.0